In [ ]:
import tensorflow as tf
import tensorflow_text as tf_text
datafiles = "/home/ericjm24/fimarch/txt/*.txt"
dataset = tf.data.TextLineDataset(tf.data.Dataset.list_files(datafiles))
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, r'''([!(),.?\-'";:])''', r' \1 '))
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, r'''[^ a-z0-9A-Z!(),.?\-'";:]''', ''))
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, r'\s+', ' '))

In [5]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 10000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

vocab = bert_vocab.bert_vocab_from_dataset(
    dataset.batch(1000).prefetch(2),
    **bert_vocab_args
)

def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

write_vocab_file('vocab.txt', vocab)

In [ ]:
WINDOW = 100
dataset = tf.strings.split(dataset, ' ')
dataset = tf_text.sliding_window(dataset, WINDOW)
dataset = tf.reshape(dataset, [-1, WINDOW])